### Libraries

In [1]:
import re
import pandas as pd
import numpy as np
import csv
import nltk
from nltk.corpus import stopwords

### Loading in Data

In [2]:
# import os
# for dirname, _, filenames in os.walk('C:/Users/eric the cool/Desktop/9665/project'):
#     print(filenames)

In [3]:
dataset = 2

In [35]:
if dataset == 1:
    train = pd.read_csv('C:/Users/eric the cool/Desktop/9665/project/SMS_train.csv', encoding = 'latin-1')
    test = pd.read_csv('C:/Users/eric the cool/Desktop/9665/project/SMS_test.csv', encoding = 'latin-1')
    train['Label'] =  train['Label'].map({"Non-Spam":0,"Spam":1})
    test['Label'] =  test['Label'].map({"Non-Spam":0,"Spam":1})

if dataset == 2:
    data = pd.read_csv('C:/Users/eric the cool/Desktop/9665/project/spam_ham_dataset.csv', encoding = 'latin-1')
    data = data.rename(columns={'label_num': 'Label', 'text': 'Message_body'})
    
    from sklearn.model_selection import train_test_split
    train, test = train_test_split(data, test_size=0.2, random_state=0)
    train = train[['Message_body', 'Label']]
    test = test[['Message_body', 'Label']]
    train = train.reset_index(drop=True)
    test = test.reset_index(drop=True)

### Preprocessing

In [30]:
def preprocessing (text):
    
    ps = nltk.stem.PorterStemmer()
    # Normalization and cleaning
    text = text.lower()
    text = re.sub("(http|https|www)(:|\.)\S+.com"," ",text)
    text = re.sub('[^a-zA-Z0-9\n]', ' ', text)
    text = re.sub("[^\w\d]"," ",text)
    text = re.sub("\d+"," ",text)
    text = re.sub('\s+',' ', text)
    text = " ".join([ps.stem(t) for t in text.split() if t not in nltk.corpus.stopwords.words("english")])
    
    # Tokenization and Stemming
#     token_text = []
#     ps = nltk.stem.PorterStemmer()
#     for word in nltk.word_tokenize(text):
#         token_text.append(ps.stem(word))
        
    return text

In [31]:
preprocessing(train['Message_body'][3])

'subject daren firm trade waha book set intern counterparti desk desk trade enrononlin follow user id password give access live price web site http www enrononlin com user id adm password welcom note case sensit pleas keep user id password secur allow transact enrononlin contact helpdesk x question problem gain access id thank stephani x'

In [40]:
processed_text = []
for i in range(train.shape[0]):
    processed_text.append(preprocessing(train.loc[i].at["Message_body"]))
    
train['processed_text'] = processed_text

In [ ]:
processed_text

In [36]:
processed_text2 = []
for i in range(test.shape[0]):
    processed_text2.append(preprocessing(test['Message_body'][i]))
    
test['processed_text'] = processed_text2

In [41]:
train.head()

,Message_body,Label,processed_text
0,"Subject: meter 986296\r\nscherlyn , per our co...",0,subject meter scherlyn per convers meter month...
1,Subject: natural gas nomination for 08 / 00\r\...,0,subject natur ga nomin enron methanol nomin fo...
2,"Subject: cleburne outage\r\ngentlemen ,\r\ni w...",0,subject cleburn outag gentlemen want clarifi l...
3,"Subject: daren ,\r\nthe firm trading waha book...",0,subject daren firm trade waha book set intern ...
4,"Subject: re : mon , 2 feb 2004 03 : 16 : 16 - ...",1,subject mon feb page load imag show view messa...


In [38]:
test.head()

,Message_body,Label,processed_text
0,Subject: ship channel hub co\r\nplease review ...,0,subject ship channel hub co pleas review comme...
1,Subject: feb 2000 intercompany accrual varianc...,0,subject feb intercompani accrual varianc list ...
2,"Subject: meter 981318\r\ndaren ,\r\nthe above ...",0,subject meter daren meter record flow jan year...
3,Subject: via - ggra is lousy mizar\r\nanti\r\n...,1,subject via ggra lousi mizar anti curb hemisph...
4,Subject: potential list - feb . 2001\r\ndaren ...,0,subject potenti list feb daren awar potenti co...


In [42]:
X_train = train['processed_text']
Y_train = train['Label']

X_test = test['processed_text']
Y_test = test['Label']


print(X_train.shape)
print(Y_train.shape)

(4136,)
(4136,)


In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer

text_vec = TfidfVectorizer(min_df=10, max_features=1000)

In [44]:
text_vec.fit(X_train.values)

train_text = text_vec.transform(X_train.values)
test_text = text_vec.transform(X_test.values)

In [45]:
print(train_text.shape)
print(test_text.shape)

(4136, 1000)
(1035, 1000)


In [46]:
import pickle
from sklearn.linear_model import LogisticRegressionCV
clf = LogisticRegressionCV(cv=5,
                           scoring ='accuracy',
                           random_state = 0,
                           n_jobs = -1,
                           verbose =3,
                           max_iter= 300).fit(train_text,Y_train)
# saved_model = open('save_model.sav','wb')
# pickle.dump(clf,saved_model)
# saved_model.close()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    5.7s remaining:    8.6s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    7.2s finished


In [ ]:
# filename ='save_model.sav'
# saved_clf = pickle.load(open(filename,'rb'))

In [47]:
print("Accuracy for the Logistic Regression is :",clf.score(train_text,Y_train))

Accuracy for the Logistic Regression is : 0.988394584139265


In [48]:
print("Accuracy for the Logistic Regression is :",clf.score(test_text,Y_test))

Accuracy for the Logistic Regression is : 0.9710144927536232
